In [138]:
import logging
import pandas as pd
import os
import json
import time

In [139]:
with open("session.txt", "r") as f:
    sessionid = f.read()

In [140]:
headers_for_html_fetching = {
            'Authority': 'www.threads.net',
            'Accept': (
                'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;'
                'q=0.8,application/signed-exchange;v=b3;q=0.7'
            ),
            'Accept-Language': 'en-US,en;q=0.9',
            'Cache-Control': 'no-cache',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Origin': 'https://www.threads.net',
            'Pragma': 'no-cache',
            'Referer': 'https://www.instagram.com',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'cross-site',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': (
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) '
                'AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15'
            ),
        }

In [141]:
import requests

def getuserfollowers(userid, after):

	url = "https://threads-api4.p.rapidapi.com/api/user/followers"

	querystring = {"user_id":userid,"end_cursor":after}

	# headers = {
	# 	"x-rapidapi-key": "33597f1514msh76f2ee2bf3e48abp12819cjsn0b3bbf51a993",
	# 	"x-rapidapi-host": "threads-api4.p.rapidapi.com"
	# }

	headers = {
		'x-rapidapi-key': "110a6c40e2msh84b7d831ac51c42p16225bjsna1e5d1776e12",
		'x-rapidapi-host': "threads-api4.p.rapidapi.com"
	}
 
	response = requests.get(url, headers=headers, params=querystring)
	if response.json()["message"].__contains__("You have exceeded the MONTHLY quota for Requests on your current plan"):
		raise Exception()
	return response.json()

# getuserfollowers("1172237643", "0")

In [142]:
import re
def get_threads_userpk(username:str):
    cookies = {
            "sessionid": sessionid
    }
    response = requests.get(
                url=f'https://www.threads.net/@{username}',
                headers=headers_for_html_fetching,
                cookies=cookies
            )
    try:
        user_id_key_value = re.search('"user_id":"(\\d+)",', response.text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

# get_threads_userpk("beiyo_")

In [143]:
dff = pd.read_csv("followers_Artificial_Intelligence.csv")
dff[dff["follower_pk"]==69655371289]#["follower_username"].unique()[0]

,Unnamed: 0.1,Unnamed: 0,user_pk,follower_pk,follower_username,follower_count


In [144]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid=sessionid)
t.retrieve_follower_by_userid_after("2018198806","0")

{'errors': [{'message': 'execution error',
   'path': ['fetch__XDTUserDict'],
   'severity': 'CRITICAL'}],
 'data': None,
 'extensions': {'is_final': True},
 'status': 'ok'}

In [145]:
# query = "Artificial Intelligence"

In [146]:

def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}
filejson = "progress2.json"

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open(filejson, "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open(filejson, "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open(filejson, "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open(filejson, "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        has_next_page = True
        while has_next_page:
            try:
                print("Pagina:", batch_number)
                # time.sleep(2)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number-1))
                # response = getuserfollowe(userid, str(25 * batch_number-1))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                if data.get("edges", []) == []:
                    return [{
                        "pk": userid,
                        "username": None,
                        "follower_count": None
                    }]
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results




def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [147]:
# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [148]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [149]:
df = pd.read_csv("followers_Artificial_Intelligence1.csv")
df['follower_pk'] = df['follower_pk'].astype(str)
unique_user_pk = df['follower_pk'].unique()
len(unique_user_pk)

3915

In [150]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "followers_followers_Artificial_Intelligence.csv"
processed_users = load_processed_users(csv_file)

In [151]:
len(processed_users)

2220

In [152]:
print(len(unique_user_pk))
unique_user_pk = unique_user_pk[unique_user_pk != "339473819"]
unique_user_pk = unique_user_pk[unique_user_pk != "1796326818"]
unique_user_pk = unique_user_pk[unique_user_pk != "16761867"]
unique_user_pk = unique_user_pk[unique_user_pk != "30850750"]
unique_user_pk = unique_user_pk[unique_user_pk != "71379428946"]
unique_user_pk = unique_user_pk[unique_user_pk != "71801041312"]
unique_user_pk = unique_user_pk[unique_user_pk != "15700501248"]
unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]

unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]
unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]
unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]
unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]
unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]
unique_user_pk = unique_user_pk[unique_user_pk != "12305626043"]
 
print(len(unique_user_pk))

3915
3907


In [153]:
def fetch_followers(pk,user_pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": user_pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"],
                "user_threads_userpk": pk
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None



follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    username= dff[dff["follower_pk"] == int(user_pk)]["follower_username"].unique()[0]
    print(user_pk, username)
    threads_user_pk = get_threads_userpk(str(username))
    results = fetch_followers(threads_user_pk,user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)   
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")


Starting follower data collection
Skipping already processed user_pk: 1451797369
Skipping already processed user_pk: 2384555054
Skipping already processed user_pk: 17096723
Skipping already processed user_pk: 55856570272
Skipping already processed user_pk: 33147008532
Skipping already processed user_pk: 3702020941
Skipping already processed user_pk: 472676306
Skipping already processed user_pk: 32185561361
Skipping already processed user_pk: 13534725
Skipping already processed user_pk: 1727625343
Skipping already processed user_pk: 8006990
Skipping already processed user_pk: 1735244853
Skipping already processed user_pk: 8960413271
Skipping already processed user_pk: 39436106246
Skipping already processed user_pk: 2343042146
Skipping already processed user_pk: 3584651501
Skipping already processed user_pk: 1594819278
Skipping already processed user_pk: 2098126112
Skipping already processed user_pk: 33063492581
Skipping already processed user_pk: 2034417720
Skipping already processed us

Exception: 

In [ ]:
df1 = pd.read_csv(csv_file)
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(2219, 4)


,follower_pk,follower_username,follower_count,user_threads_userpk
user_pk,,,,
195353,25,25,25,25
269009,50,50,50,0
301440,50,50,50,0
634084,25,25,25,25
735908,25,25,25,25
...,...,...,...,...
71695039997,1,0,0,1
71699191009,28,28,28,0
71816156861,1,0,0,1
